# Archivo 1: `model_core.py` (El Cerebro)

Este script contiene la lógica de Machine Learning: generación de datos, entrenamiento y predicción.

### 1. Importaciones y Configuración de Logs
Esta sección prepara las herramientas necesarias y configura el sistema de reportes.

In [ ]:
# Importa pandas para manejar tablas de datos (DataFrames)
import pandas as pd
# Importa numpy para operaciones matemáticas y generación de números aleatorios
import numpy as np
# Importa joblib para guardar y cargar el modelo entrenado (persistencia en disco)
import joblib
# Importa logging para mostrar mensajes de estado en la consola (info, errores)
import logging
# Importa tipos de datos para hacer el código más legible y robusto (Type Hinting)
from typing import List, Optional

# --- LIBRERÍAS DE MACHINE LEARNING (SCIKIT-LEARN) ---
# Importa el algoritmo de clasificación "Random Forest" (Bosque Aleatorio)
from sklearn.ensemble import RandomForestClassifier
# Importa Pipeline para encadenar pasos (preprocesamiento -> modelo) de forma ordenada
from sklearn.pipeline import Pipeline
# Importa StandardScaler para normalizar los datos (ponerlos en la misma escala matemática)
from sklearn.preprocessing import StandardScaler
# Importa herramienta para dividir datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split
# Importa herramientas para medir qué tan bueno es el modelo (métricas)
from sklearn.metrics import classification_report

# --- CONFIGURACIÓN DE LOGS ---
# Configura el sistema: nivel INFO y formato con fecha/hora
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# Crea un objeto 'logger' con el nombre "EmbarkCore" para identificar los mensajes de este módulo
logger = logging.getLogger("EmbarkCore")

### 2. Generación de Datos Sintéticos
Función auxiliar para crear datos falsos cuando no tenemos datos reales del juego.

In [1]:
# --- 1. FUNCIÓN DE SIMULACIÓN DE DATOS ---
def generar_datos_jugadores(n_jugadores=2000):
    """Genera datos sintéticos para entrenar el modelo si no hay datos reales."""
    
    # Fija la "semilla" aleatoria para que los resultados sean siempre iguales al probar
    np.random.seed(42)
    data = [] # Lista vacía para guardar los registros
    
    # Bucle que crea cada jugador uno por uno
    for _ in range(n_jugadores):
        # Decide aleatoriamente si es cheater (1) o no (0). 10% probabilidad de ser cheater.
        es_cheater = np.random.choice([0, 1], p=[0.90, 0.10])
        
        if es_cheater:
            # --- Perfil del Cheater ---
            # Reacción sobrehumana (media 150ms)
            time_to_dmg = np.random.normal(loc=150, scale=20) 
            # Puntería perfecta (error casi 0)
            aim_path_error = np.random.normal(loc=0.5, scale=0.5) 
            # Casi todo headshot (95%)
            headshot_rate = np.random.normal(loc=0.95, scale=0.05)
        else:
            # --- Perfil Humano ---
            # Reacción normal (media 350ms)
            time_to_dmg = np.random.normal(loc=350, scale=80) 
            # Error natural por movimiento del mouse
            aim_path_error = np.random.normal(loc=5.0, scale=2.5)
            # Headshots variados (45%)
            headshot_rate = np.random.normal(loc=0.45, scale=0.15)
            
        # Limpia el dato de headshot para que esté entre 0 y 1
        headshot_rate = min(max(headshot_rate, 0), 1)
        
        # Agrega el jugador a la lista
        data.append({
            'time_to_damage_ms': time_to_dmg,
            'aim_path_deviation': aim_path_error,
            'headshot_rate': headshot_rate,
            'is_cheater': es_cheater
        })
            
    # Retorna la tabla completa como DataFrame
    return pd.DataFrame(data)

### 3. La Clase `AntiCheatDetector`
La definición del objeto que contiene la inteligencia del sistema.

In [ ]:
# --- 2. CLASE DEL MODELO ---
class AntiCheatDetector:
    def __init__(self, n_estimators: int = 100, random_state: int = 42):
        # Define la tubería de procesamiento
        self.pipeline = Pipeline([
            ('scaler', StandardScaler()), # Paso 1: Normalizar números
            ('classifier', RandomForestClassifier( # Paso 2: Clasificar
                n_estimators=n_estimators,
                class_weight='balanced', # Importante: Prestar más atención a los cheaters (clase minoritaria)
                random_state=random_state,
                n_jobs=-1
            ))
        ])
        self.is_trained = False # Estado inicial: no entrenado
        self.version = "1.0.0"

    def train(self, df: pd.DataFrame, target_col: str, feature_cols: List[str]):
        logger.info(f"Entrenando modelo v{self.version} con {len(df)} registros...")
        
        # Separa características (X) y objetivo (y)
        X = df[feature_cols]
        y = df[target_col]
        
        # Divide en set de entrenamiento (80%) y prueba (20%)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        
        # Entrena el modelo
        self.pipeline.fit(X_train, y_train)
        self.is_trained = True
        
        # Valida y muestra el puntaje
        score = self.pipeline.score(X_test, y_test)
        logger.info(f"Modelo entrenado. Accuracy en test: {score:.4f}")

    def predict_player(self, player_stats: dict) -> dict:
        """Recibe estadísticas de UN jugador y devuelve el veredicto."""
        if not self.is_trained:
            raise Exception("Modelo no entrenado.")

        input_df = pd.DataFrame([player_stats])
        # Calcula la probabilidad de ser Cheater (Clase 1)
        probability = self.pipeline.predict_proba(input_df)[0][1] 
        
        # Aplica umbral estricto (85%) para decidir
        is_cheater = probability > 0.85
        
        return {
            "is_cheater": bool(is_cheater),
            "confidence": round(probability, 4),
            "risk_level": "CRITICAL" if probability > 0.95 else ("HIGH" if probability > 0.85 else "LOW")
        }

    def save(self, filepath: str):
        """Guarda el modelo en disco (.pkl)"""
        joblib.dump(self.pipeline, filepath)
        logger.info(f"Modelo guardado en {filepath}")

    def load(self, filepath: str):
        """Carga el modelo desde disco"""
        self.pipeline = joblib.load(filepath)
        self.is_trained = True
        logger.info(f"Modelo cargado desde {filepath}")

# Archivo 2: `api_server.py` (La API)

Este script levanta el servidor web que conecta el juego con el modelo.

### 1. Importaciones y Configuración
Esta sección importa FastAPI, define el esquema de datos (contrato) y configura la aplicación.

In [ ]:
# --- IMPORTACIONES ---
from fastapi import FastAPI, HTTPException # Framework de API y manejo de errores
from pydantic import BaseModel # Validación de datos entrantes
import os # Para manejo de archivos del sistema
import pandas as pd # Para manejo de fechas (Timestamp)
# Importa la clase del modelo desde el archivo anterior
from model_core import AntiCheatDetector, generar_datos_jugadores 

# --- INICIALIZACIÓN ---
app = FastAPI(
    title="Embark Anti-Cheat Oracle",
    description="Microservicio de detección de anomalías en FPS",
    version="1.0"
)

# --- CONTRATO DE DATOS (SCHEMA) ---
# Define qué datos DEBE enviar el juego obligatoriamente
class PlayerTelemetry(BaseModel):
    player_id: str
    time_to_damage_ms: float
    aim_path_deviation: float
    headshot_rate: float

### 2. Preparación del Modelo y Evento de Arranque
Aquí se instancia el detector y se define la lógica de "Cold Start": si no hay un modelo guardado en disco, se entrena uno nuevo automáticamente al iniciar el servidor.

In [ ]:
# --- PREPARACIÓN DEL MODELO ---
detector = AntiCheatDetector() # Instancia vacía
MODEL_PATH = "embark_model.pkl" # Ubicación del archivo del cerebro

# --- EVENTO DE ARRANQUE ---
# Se ejecuta una sola vez al encender el servidor
@app.on_event("startup")
def load_or_train_model():
    if os.path.exists(MODEL_PATH):
        # Si ya existe el archivo, cárgalo
        detector.load(MODEL_PATH)
        print("✅ Modelo pre-entrenado cargado exitosamente.")
    else:
        # Si no existe, entrena uno nuevo al momento (Cold Start)
        print("⚠️ No se encontró modelo. Entrenando uno nuevo...")
        df = generar_datos_jugadores(1000)
        detector.train(df, 'is_cheater', ['time_to_damage_ms', 'aim_path_deviation', 'headshot_rate'])
        print("✅ Modelo entrenado al vuelo.")

### 3. Endpoints (Rutas)
Definición de las rutas de la API: `/` para verificar estado y `/analyze_player` para recibir datos y devolver predicciones.

In [ ]:
# --- ENDPOINTS (RUTAS) ---

@app.get("/")
def home():
    """Ruta de prueba para ver si el servidor funciona"""
    return {"message": "Anti-Cheat System Online. Use /docs for interface."}

@app.post("/analyze_player")
def analyze_player(telemetry: PlayerTelemetry):
    """
    Recibe los datos del juego, consulta al modelo y devuelve el veredicto.
    """
    try:
        # Prepara los datos para el modelo
        input_data = {
            'time_to_damage_ms': telemetry.time_to_damage_ms,
            'aim_path_deviation': telemetry.aim_path_deviation,
            'headshot_rate': telemetry.headshot_rate
        }
        
        # Pide la predicción al cerebro (model_core)
        result = detector.predict_player(input_data)
        
        # Devuelve la respuesta JSON al cliente (Juego)
        return {
            "player_id": telemetry.player_id,
            "verdict": result,
            "timestamp": pd.Timestamp.now().isoformat()
        }
        
    except Exception as e:
        # Si algo falla, devuelve error 500
        raise HTTPException(status_code=500, detail=f"Error interno: {str(e)}")